In [1]:
import numpy as np


In [151]:
x_mean = np.array(([[0.1], [0.0]]), dtype=np.float)
x_sigma =np.array([[1, 0],[ 0, 1]]) * 3

In [152]:
np.linalg.cholesky(x_sigma)

array([[ 1.73205081,  0.        ],
       [ 0.        ,  1.73205081]])

In [153]:
def get_sigma_points_old(x_mean, x_sigma, n = 2, alpha= 0.5, beta = 2, kappa = 10):
    sigma_points = np.zeros([n, 2 * n + 1], dtype=np.float)
    
    # Use Cholesky as proxy for square root of the matrix
    L = np.linalg.cholesky(x_sigma)
    
    par_lambda = alpha * alpha * (n + kappa) - n
    sqrt_n_plus_lambda = np.sqrt(n + par_lambda)
    
    sigma_points[:, 0:1] = x_mean
    sigma_points[:, 1:n+1] = x_mean + sqrt_n_plus_lambda * L 
    sigma_points[:, n+1:2*n+1] = x_mean - sqrt_n_plus_lambda * L 
    
    w_m = np.zeros([1, 2 * n + 1 ], dtype=np.float)
    w_c = np.zeros([1, 2 * n + 1 ], dtype=np.float)
    
#     print(par_lambda)
    
    # weights w_m are for computing mean and weights w_c are
    # used for covarince calculation
    
    w_m = w_m + 1 / (2* (n + par_lambda))
    w_c = w_c + w_m
    w_m [0, 0] = par_lambda / (n + par_lambda) 
    w_c [0, 0] = w_m[0,0] + (1 - alpha * alpha + beta)
    
    return sigma_points, w_m, w_c
    
    

In [154]:
def get_sigma_points(x_mean, x_sigma, n = 2, alpha= 0.5, beta = 2, kappa = 10):
    sigma_points = np.zeros([n, 2 * n + 1], dtype=np.float)
    
    # Use Cholesky as proxy for square root of the matrix
    L = np.linalg.cholesky(x_sigma)
    
    par_lambda = alpha * alpha * (n + kappa) - n
    sqrt_n_plus_lambda = np.sqrt(n + par_lambda)
    
#     sigma_points[:, 0:1] = x_mean
#     sigma_points[:, 1:n+1] = x_mean + sqrt_n_plus_lambda * L 
#     sigma_points[:, n+1:2*n+1] = x_mean - sqrt_n_plus_lambda * L 
    
    sigma_points = []
    sigma_points.append(x_mean.T)
    gen_list=(x_mean.T + x for x in (sqrt_n_plus_lambda * L))
    sigma_points = sigma_points + list(gen_list)
    gen_list=(x_mean.T - x for x in (sqrt_n_plus_lambda * L))
    sigma_points = sigma_points + list(gen_list)
    
    w_m = np.zeros([ 2 * n + 1, 1 ], dtype=np.float)
    w_c = np.zeros([ 2 * n + 1, 1 ], dtype=np.float)
    
#     print(par_lambda)
    
    # weights w_m are for computing mean and weights w_c are
    # used for covarince calculation
    
    w_m = w_m + 1 / (2* (n + par_lambda))
    w_c = w_c + w_m
    w_m [0, 0] = par_lambda / (n + par_lambda) 
    w_c [0, 0] = w_m[0,0] + (1 - alpha * alpha + beta)
    
    return sigma_points, w_m, w_c
    

In [155]:
n = 2
alpha= 0.5
beta = 2
kappa = 10
par_lambda = alpha * alpha * (n + kappa) - n
sqrt_n_plus_lambda = np.sqrt(n + par_lambda)

L = np.linalg.cholesky(x_sigma)
sigma_points = []
sigma_points.append(x_mean.T)
gen_list=(x_mean.T + x for x in (sqrt_n_plus_lambda * L))
sigma_points = sigma_points + list(gen_list)
gen_list=(x_mean.T - x for x in (sqrt_n_plus_lambda * L))
sigma_points = sigma_points + list(gen_list)

In [156]:
w_m = np.ones([ 2 * n + 1 , 1], dtype=np.float)
w_c = np.zeros([1, 2 * n + 1 ], dtype=np.float)
    
#     print(par_lambda)
    
    # weights w_m are for computing mean and weights w_c are
    # used for covarince calculation
    
w_m = w_m / (2* (n + par_lambda))
w_c = w_m.copy()
w_m_list = w_m.tolist()
w_c_list = w_c.tolist()
w_m_list[0] = par_lambda / (n + par_lambda) 
w_c_list[0] = w_m[0] + (1 - alpha * alpha + beta)
    

In [157]:
sum(w_m_list)

TypeError: unsupported operand type(s) for +: 'float' and 'list'

In [158]:
sum(w_c_list)

array([ 3.58333333])

In [159]:
for item in (sqrt_n_plus_lambda * L):
    print(1)
    print(item + x_mean.T)

1
[[ 3.1  0. ]]
1
[[ 0.1  3. ]]


In [160]:
list(map(f, sigma_points))

[array([[ 1.2,  1. ]]),
 array([[ 7.2,  1. ]]),
 array([[ 1.2,  7. ]]),
 array([[-4.8,  1. ]]),
 array([[ 1.2, -5. ]])]

In [161]:
len(sigma_points)

5

In [162]:
def f(x):
    x = np.array(x, dtype=np.float)
    x2 = 2 * x + 1
    return x2

In [171]:
def unscented_transform(f, x_mean, x_sigma):
    n = 2
    sigma_points, w_m, w_c = get_sigma_points(x_mean, x_sigma, n=n)
    
    transformed_points = list(map(f,sigma_points))
    
    weighted_samples = [weight*sample for weight, sample in zip(w_m, transformed_points)]
    pred_mean = sum (weighted_samples)
    pred_mean = pred_mean.reshape([1, n])
    gofx_minus_mean = transformed_points - pred_mean
#     p_s = np.matmul( gofx_minus_mean, np.transpose(gofx_minus_mean))
    #pred_sigma = np.sum(np.multiply( np.matmul( gofx_minus_mean, gofx_minus_mean.transpose()) , w_m ))
    return pred_mean#, p_s

In [172]:
sigma_points, w_m, w_c = get_sigma_points(x_mean, x_sigma)

In [173]:
f(sigma_points)

array([[[ 1.2,  1. ]],

       [[ 7.2,  1. ]],

       [[ 1.2,  7. ]],

       [[-4.8,  1. ]],

       [[ 1.2, -5. ]]])

In [174]:
len(w_m.T)

1

In [175]:
for weight, sample in zip(w_m.T, sigma_points):
    print(1)
    print(weight)
    print(sample)

1
[ 0.33333333  0.16666667  0.16666667  0.16666667  0.16666667]
[[ 0.1  0. ]]


In [176]:
unscented_transform(f, x_mean, x_sigma)

array([[ 1.2,  1. ]])

In [169]:
f(x_mean)

array([[ 1.2],
       [ 1. ]])

In [170]:
x_mean

array([[ 0.1],
       [ 0. ]])